# Seq-to-first-iso

> Compute first two isotopologues intensities from sequences

**seq-to-first-iso** is a tool to compute isotopologue intentities M0 and M1 of peptide sequences with Natural Carbon
and with 99.99 % 12C enriched carbon.  
The program can take into account unlabelled amino acids to simulate auxotrophies to amino acids.

---
## Using the Command-Line Interface

*Note: the exclamation marks "!" are used for the notebook, in a real terminal, you will not need them.*

In [6]:
!seq-to-first-iso -h

usage: seq-to-first-iso [-h] [-o OUTPUT] [-n amino_a] input

Read a file of sequences and creates a tsv file

positional arguments:
  input                 file to parse

optional arguments:
  -h, --help            show this help message and exit
  -o OUTPUT, --output OUTPUT
                        name of output file
  -n amino_a, --non-labelled-aa amino_a
                        amino acids with default abundance


The output file will have columns

|sequence| mass| formula|formula_X| M0_NC| M1_NC| M0_12C| M1_12C|
|--------|-----|--------|---------|------|------|-------|-------|
original sequence|sequence mass|chemical formula|chemical formula with X|M0 in NC|M1 in NC|M0 in 12C|M1 in 12C

X: Virtual element created to represent unlabelled carbon  
NC: Normal Condition (Natural Carbon)  
12C: 12C condition (12C enriched carbon)    

In [75]:
# File used.
!cat peptides.txt

YAQEISR
VGFPVLSVKEHK
LAMVIIKEFVDDLK


### Minimal command

In [51]:
!seq-to-first-iso peptides.txt

[2019-06-26, 12:09:01] INFO    : Parsing file
[2019-06-26, 12:09:01] INFO    : Computing formula
[2019-06-26, 12:09:01] INFO    : Computing composition of modifications
[2019-06-26, 12:09:01] INFO    : Computing mass
[2019-06-26, 12:09:01] INFO    : Computing M0 and M1


Running the command above will create a file with tab-separated values : *peptides_stfi.tsv*

In [52]:
!column -t peptides_stfi.tsv

sequence        mass              formula          formula_X        M0_NC                M1_NC                M0_12C              M1_12C
YAQEISR         865.42938099921   C37H59O13N11     C37H59O13N11     0.6206414140575179   0.280870823368276    0.9206561231798033  0.05161907174495234
VGFPVLSVKEHK    1338.7659712609   C63H102O16N16    C63H102O16N16    0.4550358985377136   0.34506032928190855  0.8905224988642593  0.07411308335404865
LAMVIIKEFVDDLK  1632.91606619252  C76H128O21N16S1  C76H128O21N16S1  0.36994021481230627  0.3373188347614264   0.8315762004558261  0.08101653544902196


### Changing output name

You can also change the name of the output file

In [65]:
!seq-to-first-iso peptides.txt -o sequence

[2019-06-26, 12:15:51] INFO    : Parsing file
[2019-06-26, 12:15:51] INFO    : Computing formula
[2019-06-26, 12:15:51] INFO    : Computing composition of modifications
[2019-06-26, 12:15:51] INFO    : Computing mass
[2019-06-26, 12:15:51] INFO    : Computing M0 and M1


In [66]:
!column -t sequence.tsv

sequence        mass              formula          formula_X        M0_NC                M1_NC                M0_12C              M1_12C
YAQEISR         865.42938099921   C37H59O13N11     C37H59O13N11     0.6206414140575179   0.280870823368276    0.9206561231798033  0.05161907174495234
VGFPVLSVKEHK    1338.7659712609   C63H102O16N16    C63H102O16N16    0.4550358985377136   0.34506032928190855  0.8905224988642593  0.07411308335404865
LAMVIIKEFVDDLK  1632.91606619252  C76H128O21N16S1  C76H128O21N16S1  0.36994021481230627  0.3373188347614264   0.8315762004558261  0.08101653544902196


### Choosing unlabelled amino acids

In [63]:
!seq-to-first-iso peptides.txt -n V,W -o sequence

[2019-06-26, 12:11:16] INFO    : Amino acid with default abundance: ['V', 'W']
[2019-06-26, 12:11:16] INFO    : Parsing file
[2019-06-26, 12:11:16] INFO    : Computing formula
[2019-06-26, 12:11:16] INFO    : Computing composition of modifications
[2019-06-26, 12:11:16] INFO    : Computing mass
[2019-06-26, 12:11:16] INFO    : Computing M0 and M1


In [64]:
!column -t sequence.tsv

sequence        mass              formula          formula_X           M0_NC                M1_NC               M0_12C              M1_12C
YAQEISR         865.42938099921   C37H59O13N11     C37H59O13N11        0.6206414140575179   0.280870823368276   0.9206561231798033  0.05161907174495234
VGFPVLSVKEHK    1338.7659712609   C63H102O16N16    C48H102O16N16X15    0.45503589853771365  0.3450603292819086  0.7589558393662944  0.18515489894512063
LAMVIIKEFVDDLK  1632.91606619252  C76H128O21N16S1  C66H128O21N16S1X10  0.36994021481230627  0.3373188347614264  0.7475090558698947  0.15292723586285323


The carbon of unlabelled amino acids is shown as X in column "formula_X".  
We can observe that for sequence "YAQEISR" that has no unlabelled amino acids, M0 and M1 are the same as the previous *sequence.tsv*, regardless of the condition.  
In contrast sequence "VGFPVLSVKEHK", in 12C condition, has M0 go down from 0.8905224988642593 to 0.7589558393662944 and M1 go up from 0.07411308335404865 to 0.18515489894512063.

---
## Using the API

You can also use seq-to-first-iso's functions by importing the module seq_to_first_iso

In [27]:
from pprint import pprint

import pandas as pd
from pyteomics import mass

import seq_to_first_iso as stfi

In [47]:
# Variables used for showcase.
peptide_seq = "YAQEISRAR"
unlabelled_amino_acids = ["A", "R"]

### Abundances defined in seq-to-first-iso

In [12]:
isotopic_abundance = stfi.isotopic_abundance
C12_abundance = stfi.C12_abundance

In [95]:
pprint(isotopic_abundance)

{'C[12]': 0.9893,
 'C[13]': 0.0107,
 'H[1]': 0.999885,
 'H[2]': 0.000115,
 'N[14]': 0.99632,
 'N[15]': 0.00368,
 'O[16]': 0.99757,
 'O[17]': 0.00038,
 'O[18]': 0.00205,
 'S[32]': 0.9493,
 'S[33]': 0.0076,
 'S[34]': 0.0429,
 'X[12]': 0.9893,
 'X[13]': 0.0107}


In [26]:
pprint(C12_abundance)

{'C[12]': 0.9999,
 'C[13]': 9.999999999998899e-05,
 'H[1]': 0.999885,
 'H[2]': 0.000115,
 'N[14]': 0.99632,
 'N[15]': 0.00368,
 'O[16]': 0.99757,
 'O[17]': 0.00038,
 'O[18]': 0.00205,
 'S[32]': 0.9493,
 'S[33]': 0.0076,
 'S[34]': 0.0429,
 'X[12]': 0.9893,
 'X[13]': 0.0107}


isotopic_abundance and C12_abundance are dictionaries with abundances of common isotopes of organic elements.  
C12_abundance has a 12C abundance of 99.99 %, hence 13C abundance is 0.01 %.  
Element X is a virtual element created to replace the carbon of unlabelled amino acids, it has the same isotopic abundances as natural carbon.

### Separate sequences according to unlabelled amino acids

In [45]:
help(stfi.separate_labelled)

Help on function separate_labelled in module seq_to_first_iso.seq_to_first_iso:

separate_labelled(sequence, unlabelled_aa)
    Get the sequence of unlabelled amino acids from a sequence.
    
    Parameters
    ----------
    sequence : str
        String of amino acids
    unlabelled_aa
        Container (list, string...) of unlabelled amino acids
    
    Returns
    -------
    tuple(str, str)
        The sequences as a tuple of string with:
        - the sequence without the unlabelled amino acids
        - the unlabelled amino acids in the sequence



In [93]:
# Separate sequence "YAQEISRAR" with amino acids A and R unlabelled.
labelled_sequence, unlabelled_sequence = stfi.separate_labelled(peptide_seq, unlabelled_aa=unlabelled_amino_acids)
print(f"sequence with labelled carbon: {labelled_sequence}\n"
      f"sequence with unlabelled carbon: {unlabelled_sequence}")

sequence with labelled carbon : YQEIS
sequence with unlabelled carbon : ARAR


### Obtain a composition with element X

In [44]:
help(stfi.seq_to_midas)

Help on function seq_to_midas in module seq_to_first_iso.seq_to_first_iso:

seq_to_midas(sequence_l, sequence_nl)
    Take 2 amino acid sequences and return the composition for MIDAs.
    
    Parameters
    ----------
    sequence_l : str
        Sequence with labelled amino acids.
    sequence_nl : str
        Sequence where amino acids are not labelled.
    
    Notes
    -----
    The function assumes the second sequence has no terminii.



In [49]:
# Get the chemical formula with unlabelled carbon as element X.
chem_formula = stfi.seq_to_midas(labelled_sequence, unlabelled_sequence)
print(f"Composition of {peptide_seq} with {unlabelled_amino_acids} unlabelled:\n{chem_formula}")

Composition of YAQEISRAR with ['A', 'R'] unlabelled:
Composition({'H': 76, 'C': 28, 'O': 15, 'N': 16, 'X': 18})


In [ ]:
# If all amino acids are labelled, you can pass an empty string.
labelled_formula = stfi.seq_to_midas(labelled_sequence, "")
print(f"Composition of {peptide_seq} with {unlabelled_amino_acids} unlabelled:\n{labelled_formula}")

### Compute isotopologue intensity

In [43]:
help(stfi.compute_M0_nl)
print("-" * 79)
help(stfi.compute_M1_nl)

Help on function compute_M0_nl in module seq_to_first_iso.seq_to_first_iso:

compute_M0_nl(f, a)
    Return the monoisotopic abundance M0 of a formula with mixed labels.
    
    Parameters
    ----------
    f : pyteomics.mass.Composition
        Chemical formula, as a dict of counts for each element:
        {element_name: count_of_element_in_sequence, ...}.
    a : dict
        Dictionary of abundances of isotopes, in the format:
        {element_name[isotope_number]: relative abundance, ..}.
    
    Returns:
    float
        Value of M0.
    
    Notes
    -----
    X represents C with default isotopic abundance.

-------------------------------------------------------------------------------
Help on function compute_M1_nl in module seq_to_first_iso.seq_to_first_iso:

compute_M1_nl(f, a)
    Compute abundance of second isotopologue M1 from its formula.
    
    Parameters
    ----------
    f : pyteomics.mass.Composition
        Chemical formula, as a dict of counts for each elem

In [37]:
# Compute M0 with natural carbon.
first_isotopologue = stfi.compute_M0_nl(chem_formula, isotopic_abundance)
print(f"M0 in Normal Condition: {first_isotopologue}")

first_isotopologue = stfi.compute_M0_nl(chem_formula, C12_abundance)
print(f"M0 in 12C condition: {first_isotopologue}")

M0 in Normal Condition : 0.6206414140575179
M0 in 12C condition : 0.9206561231798033


In [38]:
# Compute M1 with natural carbon.
first_isotopologue = stfi.compute_M1_nl(chem_formula, isotopic_abundance)
print(f"M1 in Normal Condition: {first_isotopologue}")

first_isotopologue = stfi.compute_M1_nl(chem_formula, C12_abundance)
print(f"M1 in 12C condition: {first_isotopologue}")

M1 in Normal Condition : 0.280870823368276
M1 in 12C condition : 0.05161907174495234


### Parse a file with peptide sequences

seq-to-first-iso accepts files with 1 sequence per line.  
Optionally, annotations/sequence IDs can be placed in the same line before sequences if separated by a separator (default: "\t"). The program declares that the file has annotations if the separator is found on the first line.   
The parser will ignore lines where sequences have incorrect characters (not in "ACDEFGHIKLMNPQRSTVWY") unless it corresponds to XTandem's Post-Translational Modification notation.

In [76]:
help(stfi.sequence_parser)

Help on function sequence_parser in module seq_to_first_iso.seq_to_first_iso:

sequence_parser(file, sep='\t')
    Return information on sequences parsed from a file.
    
    Parameters
    ----------
    file : str
        Filename, the file can either just have sequences for each line or
        can have have annotations and sequences with a separator in-between.
    sep : str, optional
        Separator for files with annotations.
    
    Returns
    -------
    dict
        Parsed output with "key: values" :
        - "annotations": a list of annotations if any.
        - "raw_sequences": a list of unmodified peptide sequences.
        - "sequences": a list of uppercase peptide sequences.
        - "modifications": a list of lists of PTMs.
        - "ignored_lines": and the number of ignored lines.
    
    Warnings
    --------
    The function uses the first line to evaluate if the file has
    annotations or not, hence a file should have a consistent format.
    
    Notes
   

In [77]:
parsed_output = stfi.sequence_parser("peptides.txt")
pprint(parsed_output)

{'annotations': [],
 'ignored_lines': 0,
 'modifications': [[], [], []],
 'raw_sequences': ['YAQEISR', 'VGFPVLSVKEHK', 'LAMVIIKEFVDDLK'],
 'sequences': ['YAQEISR', 'VGFPVLSVKEHK', 'LAMVIIKEFVDDLK']}


For *peptides.txt*, the list of annotations is empty (there are no annotations), no lines are ignored and 
no modifications were found so raw_sequences (with modifications) are the same as sequences (without modifications)

In [80]:
# Get the values in the returned dict.
annotations = parsed_output["annotations"]
ignored_lines = parsed_output["ignored_lines"]
modifications = parsed_output["modifications"]
sequences = parsed_output["sequences"]
raw_sequences = parsed_output["raw_sequences"]

### Create a dataframe from a list of sequences

In [78]:
help(stfi.seq_to_tsv)

Help on function seq_to_tsv in module seq_to_first_iso.seq_to_first_iso:

seq_to_tsv(sequences, unlabelled_aa, **kwargs)
    Create a dataframe from sequences and return its name.
    
    Parameters
    ----------
    sequences : list of str
        List of pure peptide sequences string.
    unlabelled_aa : container object
        Container of unlabelled amino acids.
    **kwargs
        Optional keyword arguments:
            annotations : list of str, optional
                 List of IDs for the sequences.
            raw_sequences : list of str, optional
                 List of sequences with Xtandem PTMs.
            modifications : list of str, optional
                 List of modifications for raw_sequences.
    
    Returns
    -------
    pandas.Dataframe
        Dataframe with : annotation (optional), sequence, mass,
                         formula, formula_X, M0_NC, M1_NC, M0_12C, M1_12C.
    
    Warnings
    --------
    If raw_sequence is provided, modifications must

In [81]:
# The dict returned by sequence_parser can be unpacked in the function (remove "ignored_lines" against warning).
df_peptides = stfi.seq_to_tsv(sequences, unlabelled_aa=unlabelled_amino_acids)
df_peptides.head()

[2019-06-26, 14:04:53] INFO    : Computing formula
[2019-06-26, 14:04:53] INFO    : Computing mass
[2019-06-26, 14:04:53] INFO    : Computing M0 and M1


,sequence,mass,formula,formula_X,M0_NC,M1_NC,M0_12C,M1_12C
0,YAQEISR,865.429381,C37H59O13N11,C28H59O13N11X9,0.620641,0.280871,0.836451,0.127566
1,VGFPVLSVKEHK,1338.765971,C63H102O16N16,C63H102O16N16,0.455036,0.345060,0.890522,0.074113
2,LAMVIIKEFVDDLK,1632.916066,C76H128O21N16S1,C73H128O21N16S1X3,0.369940,0.337319,0.805409,0.104359


In [82]:
# Dataframe with annotations.
mock_annotations = ["id1", "id2", "id3"]
df_peptides = stfi.seq_to_tsv(sequences, unlabelled_aa=unlabelled_amino_acids, annotations=mock_annotations)
df_peptides.head()

[2019-06-26, 14:10:00] INFO    : Computing formula
[2019-06-26, 14:10:00] INFO    : Computing mass
[2019-06-26, 14:10:00] INFO    : Computing M0 and M1


,annotation,sequence,mass,formula,formula_X,M0_NC,M1_NC,M0_12C,M1_12C
0,id1,YAQEISR,865.429381,C37H59O13N11,C28H59O13N11X9,0.620641,0.280871,0.836451,0.127566
1,id2,VGFPVLSVKEHK,1338.765971,C63H102O16N16,C63H102O16N16,0.455036,0.345060,0.890522,0.074113
2,id3,LAMVIIKEFVDDLK,1632.916066,C76H128O21N16S1,C73H128O21N16S1X3,0.369940,0.337319,0.805409,0.104359


### Get the composition of a list of modifications

In [83]:
help(stfi.get_mods_composition)

Help on function get_mods_composition in module seq_to_first_iso.seq_to_first_iso:

get_mods_composition(modifications)
    Return the composition of a list of modifications.
    
    Parameters
    ----------
    modifications: list of str
        List of modifications string (corresponding to Unimod titles).
    
    Returns
    -------
    pyteomics.mass.Composition
        The total composition change.
    
    ???: Have the mass.Unimod() dict as parameter ?



In [87]:
# Modifications must be strict Unimod entries title.
modification_list = ["Acetyl", "Phospho", "phospho"]  # phospho does not correspond to a title, it will be ignored
total_composition = stfi.get_mods_composition(modification_list)
print(f"Total composition shift for {modification_list} is {total_composition}")

[2019-06-26, 14:18:02] WARNING : Unimod entry not found for : phospho


Total composition shift for ['Acetyl', 'Phospho', 'phospho'] is Composition({'H': 3, 'C': 2, 'O': 4, 'P': 1})


### Get human-readable chemical formula

In [88]:
help(stfi.formula_to_str)

Help on function formula_to_str in module seq_to_first_iso.seq_to_first_iso:

formula_to_str(composition)
    Return formula from Composition as a string.
    
    Parameters
    ----------
    composition : pyteomics.mass.Composition
        Chemical formula.
    
    Returns
    -------
    str
        Human-readable string of the formula.
    
    Warnings
    --------
    If the composition has elements not in USED_ELEMS, they will not
    be added to the output.



In [89]:
# This is the function used to get the formulas in the output.
formula_str = stfi.formula_to_str(total_composition)
print(f"{total_composition} becomes {formula_str}")

Composition({'H': 3, 'C': 2, 'O': 4, 'P': 1}) becomes C2H3O4P1


In [91]:
# !!! Warning: if the Composition has elements not in "CHONPSX", they will not be in the final string.
bad_composition = mass.Composition("U")
formula_str = stfi.formula_to_str(bad_composition)
print(f"{bad_composition} becomes {formula_str}")

Composition({'H': 7, 'C': 3, 'O': 2, 'N': 1, 'Se': 1}) becomes C3H7O2N1


Here, "non-CHONPSX" element **Se (Selenium) is ignored**.